In [1]:
# From https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/notebooks/eager_basics.ipynb

In [2]:
# Eager execution enables a more interactive frontend to TensorFlow
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
# Tensor operations.
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


In [4]:
# Each tensor has a shape and datatype
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


In [5]:
# Most obvious differences between NumPy arrays and TensorFlow Tensors are:
# 1. Tensors can be backed by accelerator memory (GPU, TPU)
# 2. Tensors are immutable

In [7]:
# Conversion between Tensors and NumPy ndarrays is simple.  both automatically convert.
# Tensors can be explicitly converted to NumPy ndarrays by invoking .numpy()
# NOTE: Tensors may be running on GPU memory and need to convert to host
# memory if converting to NumPy ndarrays.

import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [8]:
# Without annotations, TensorFlow automatically decides whether to use GPU or CPU.
x = tf.random_uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
False
Is the Tensor on GPU #0:  
False


In [9]:
# Tensor.device provides a fully qualified string name of device hosting
# contents of Tensor.

# You can use tf.device context manager to run on specific device.
def time_matmul(x):
  %timeit tf.matmul(x, x)

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random_uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.test.is_gpu_available():
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
29.5 ms ± 384 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Datasets

In [12]:
# Uses tf.data.Dataset API to build pipelines to feed data to model.
# This covers:
# - Creating a Dataset
# - Iteration over a Dataset with eager execution enabled.

# Datasets API recommended for building performant, complex input
# pipelines from simple. re-usable pieces that will feed your model's
# training or evaluation loops.

# Using Eager Execution, iterating over elements of datasets is easier.
# You can use Python iteration over tf.data.Dataset object instead of
# having to explicitly create an tf.data.Iterator object.

# from_tensor_slices creates a Dataset whose elements are
# slices of the given tensors.  This can be inefficient if 
# large dataset and Eager Execution is not enabled.
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

# TextLineDataset is a dataset comprising lines from one or more text files.
ds_file = tf.data.TextLineDataset(filename)

In [13]:
# Use transformations functions like map, batch, shuffle, etc. to
# apply transformations to the records of the dataset.

ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [14]:
# When eager execution is enabled, Dataset objects support iteration.

print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 16], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
